In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import datetime
import math, time
import sklearn
import sklearn.preprocessing
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.layers import Dense, Bidirectional
from keras.layers import LSTM
from keras.preprocessing import sequence
import csv 
import os.path
import gc
import time
import pickle

C:\Users\Olegok\Anaconda3\envs\py36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def create_dataset(seq_len = 20):
    data = pd.read_csv('stocks/AAPL.csv')
    data = data.drop(['direction_up', 'direction_down','Date', 'direction'], axis=1)
    data = data.dropna(axis=0, how='any')
    min_max_scaler = sklearn.preprocessing.MinMaxScaler()
    for key in data:
        data[key] = min_max_scaler.fit_transform(data[key].values.reshape(-1,1))
    x_data = data.as_matrix()
    x_data_seq = []
    y_data_seq = []
    for index in range(len(data) - seq_len): 
        x_data_seq.append(x_data[index: index + seq_len])
        y_data_seq.append(x_data[index + seq_len])
    x_train = np.array(x_data_seq[:math.ceil(len(data)*0.8)][:][:])
    y_train = np.array(y_data_seq[:math.ceil(len(data)*0.8)][:])
    x_test = np.array(x_data_seq[math.ceil(len(data)*0.8):][:][:])
    y_test = np.array(y_data_seq[math.ceil(len(data)*0.8):][:])
    return x_train, y_train, x_test, y_test

In [3]:
def Model(_batch_size, _n_neurons=128 , _dropout=0.5, seq_len=20, _activation='relu'):
    model = Sequential()
    model.add(LSTM(_n_neurons, batch_input_shape=(_batch_size, seq_len, 36)))
    model.add(Dropout(_dropout))
    #model.add(Flatten())
    model.add(Dense(36, activation=_activation)) #relu & signoid
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [4]:
def fit_dataset(batch_size, x_train, y_train, x_test, y_test): 
    x_train = x_train[:(x_train.shape[0]//batch_size)*batch_size]
    y_train = y_train[:(y_train.shape[0]//batch_size)*batch_size]
    x_test = x_test[:(x_test.shape[0]//batch_size)*batch_size]
    y_test = y_test[:(y_test.shape[0]//batch_size)*batch_size]
    return x_train, y_train, x_test, y_test

In [5]:
n_epoch=35
n_neurons = 350
file_name = r'activation_tune.csv'
for activation in ['softmax','elu', 'selu', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear',
                         'LeakyReLU', 'PReLU', 'ELU', 'ThresholdedReLU', 'Softmax']:
    batch = 10
    data = []
    start_time = time.time()
    model = Model(batch,n_neurons, 0.5, 20, activation)
    x_train, y_train, x_test, y_test = create_dataset()
    x_train, y_train, x_test, y_test = fit_dataset(batch, x_train, y_train, x_test, y_test)
    history = model.fit(x_train,
         y_train,
        batch_size=batch,
         epochs=n_epoch,
       validation_data=(x_test, y_test),
        verbose=1)
    score = model.evaluate(x_test, y_test, batch_size=batch)
    data.append(score)
    data.append(time.time() - start_time)
    data.append(activation)
    
    filehandler = open('tryin-4-'+activation+'.p', 'wb')
    hist = pd.DataFrame(data=[history.history["loss"], history.history["val_loss"]])
    pickle.dump(hist, filehandler)
    filehandler.close()
    
    del model
    with open(file_name, 'a', newline='') as f:
        writer = csv.writer(f)
        #writer.writerow(['Time', 'Score', 'activation'])
        writer.writerow(data)

Train on 7510 samples, validate on 1850 samples
Epoch 1/1
1850/1850 [==============================] - 3s 2ms/step
Train on 7510 samples, validate on 1850 samples
Epoch 1/1
1850/1850 [==============================] - 3s 1ms/step
Train on 7510 samples, validate on 1850 samples
Epoch 1/1
1850/1850 [==============================] - 3s 2ms/step
Train on 7510 samples, validate on 1850 samples
Epoch 1/1
1850/1850 [==============================] - 3s 2ms/step
Train on 7510 samples, validate on 1850 samples
Epoch 1/1
1850/1850 [==============================] - 3s 2ms/step
Train on 7510 samples, validate on 1850 samples
Epoch 1/1
1850/1850 [==============================] - 3s 2ms/step
Train on 7510 samples, validate on 1850 samples
Epoch 1/1
1850/1850 [==============================] - 3s 1ms/step
Train on 7510 samples, validate on 1850 samples
Epoch 1/1
1850/1850 [==============================] - 3s 2ms/step


KeyboardInterrupt: 